In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Uploading Dataset and inspecting its contents
df = pd.DataFrame()
df = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/Video_Games_Sales_as_at_22_Dec_2016.csv', index_col=0)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
Name,,,,,,,,,,,,,,,
Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Dropped all missing values
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6825 entries, Wii Sports to STORM: Frontline Nation
Data columns (total 15 columns):
Platform           6825 non-null object
Year_of_Release    6825 non-null float64
Genre              6825 non-null object
Publisher          6825 non-null object
NA_Sales           6825 non-null float64
EU_Sales           6825 non-null float64
JP_Sales           6825 non-null float64
Other_Sales        6825 non-null float64
Global_Sales       6825 non-null float64
Critic_Score       6825 non-null float64
Critic_Count       6825 non-null float64
User_Score         6825 non-null object
User_Count         6825 non-null float64
Developer          6825 non-null object
Rating             6825 non-null object
dtypes: float64(9), object(6)
memory usage: 853.1+ KB


In [4]:
# Changed Release Year to integers
df['Year_of_Release'] = (df['Year_of_Release']).astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
Name,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E


In [5]:
# Seeing how many types of genres there are
df['Genre'].value_counts()

Action          1630
Sports           943
Shooter          864
Role-Playing     712
Racing           581
Platform         403
Misc             384
Fighting         378
Simulation       297
Strategy         267
Adventure        248
Puzzle           118
Name: Genre, dtype: int64

In [6]:
# Making columns for each type of game
df['Action'] = (df['Genre'] == 'Action').astype(int)
df['Adventure'] = (df['Genre'] == 'Adventure').astype(int)
df['Fighter'] = (df['Genre'] == 'Fighting').astype(int)
df['Platformer'] = (df['Genre'] == 'Platform').astype(int)
df['Puzzler'] = (df['Genre'] == 'Puzzle').astype(int)
df['Racing'] = (df['Genre'] == 'Racing').astype(int)
df['RPG'] = (df['Genre'] == 'Role-Playing').astype(int)
df['Shooter'] = (df['Genre'] == 'Shooter').astype(int)
df['Simulation'] = (df['Genre'] == 'Simulation').astype(int)
df['Sports'] = (df['Genre'] == 'Sports').astype(int)
df['Strategy'] = (df['Genre'] == 'Strategy').astype(int)
df['Variety'] = (df['Genre'] == 'Misc').astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,...,Fighter,Platformer,Puzzler,Racing,RPG,Shooter,Simulation,Sports,Strategy,Variety
Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,...,0,0,0,0,0,0,0,1,0,0
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,...,0,0,0,1,0,0,0,0,0,0
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,...,0,0,0,0,0,0,0,1,0,0
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,...,0,1,0,0,0,0,0,0,0,0
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
# Seeing how many types of consoles there are
df['Platform'].value_counts()

PS2     1140
X360     858
PS3      769
PC       651
XB       565
Wii      479
DS       464
PSP      390
GC       348
PS4      239
GBA      237
XOne     159
3DS      155
PS       150
PSV      118
WiiU      89
DC        14
Name: Platform, dtype: int64

In [8]:
# Making columns for some of the consoles
df['Game Boy Advance'] = (df['Platform'] == 'GBA').astype(int)
df['Nintendo 3DS'] = (df['Platform'] == '3DS').astype(int)
df['Nintendo DS'] = (df['Platform'] == 'DS').astype(int)
df['Nintendo Gamecube'] = (df['Platform'] == 'GC').astype(int)
df['Nintendo Wii'] = (df['Platform'] == 'Wii').astype(int)
df['Nintendo Wii U'] = (df['Platform'] == 'WiiU').astype(int)
df['PC'] = (df['Platform'] == 'PC').astype(int)
df['Playstation'] = (df['Platform'] == 'PS').astype(int)
df['Playstation 2'] = (df['Platform'] == 'PS2').astype(int)
df['Playstation 3'] = (df['Platform'] == 'PS3').astype(int)
df['Playstation 4'] = (df['Platform'] == 'PS4').astype(int)
df['Playstation Vita'] = (df['Platform'] == 'PSV').astype(int)
df['PSP'] = (df['Platform'] == 'PSP').astype(int)
df['Sega Dreamcast'] = (df['Platform'] == 'DC').astype(int)
df['XBOX'] = (df['Platform'] == 'XB').astype(int)
df['XBOX 360'] = (df['Platform'] == 'X360').astype(int)
df['XBOX One'] = (df['Platform'] == 'XOne').astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,...,Playstation,Playstation 2,Playstation 3,Playstation 4,Playstation Vita,PSP,Sega Dreamcast,XBOX,XBOX 360,XBOX One
Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,...,0,0,0,0,0,0,0,0,0,0
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,...,0,0,0,0,0,0,0,0,0,0
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,...,0,0,0,0,0,0,0,0,0,0
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,...,0,0,0,0,0,0,0,0,0,0
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Uploading second Dataset and turning the JSON object into a DataFrame
json_vg = '/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/reviews_Video_Games_5.json'

# Reading the entire file into a python array
with open(json_vg, 'rb') as f:
    data = f.readlines()

# Removing the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# Turning everything into one JSON object
data_vg = "[" + ','.join(data) + "]"

df_2 = pd.read_json(data_vg)

In [10]:
# Inspecting its contents
df_2.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0700099867,"[8, 12]",1,Installing the game was a struggle (because of...,"07 9, 2012",A2HD75EMZR8QLN,123,Pay to unlock content? I don't think so.,1341792000
1,0700099867,"[0, 0]",4,If you like rally cars get this game you will ...,"06 30, 2013",A3UR8NLLY1ZHCX,"Alejandro Henao ""Electronic Junky""",Good rally game,1372550400
2,0700099867,"[0, 0]",1,1st shipment received a book instead of the ga...,"06 28, 2014",A1INA0F5CWW3J4,"Amazon Shopper ""Mr.Repsol""",Wrong key,1403913600
3,0700099867,"[7, 10]",3,"I got this version instead of the PS3 version,...","09 14, 2011",A1DLMTOTHQ4AST,ampgreen,"awesome game, if it did not crash frequently !!",1315958400
4,0700099867,"[2, 2]",4,I had Dirt 2 on Xbox 360 and it was an okay ga...,"06 14, 2011",A361M14PU2GUEG,"Angry Ryan ""Ryan A. Forrest""",DIRT 3,1308009600


In [11]:
# Dropped all missing values
df_2 = df_2.dropna()
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228967 entries, 0 to 231779
Data columns (total 9 columns):
asin              228967 non-null object
helpful           228967 non-null object
overall           228967 non-null int64
reviewText        228967 non-null object
reviewTime        228967 non-null object
reviewerID        228967 non-null object
reviewerName      228967 non-null object
summary           228967 non-null object
unixReviewTime    228967 non-null int64
dtypes: int64(2), object(7)
memory usage: 17.5+ MB


In [13]:
# Function to tokenize words from Amazon Reviews
def tokenizer(x):
    """Tokenizes words, removes stops words and counts the tokens"""
    x = x.str.lower()
    w_tokens = x.apply(word_tokenize)
    sr = stopwords.words('english')
    for token in w_tokens:
        for string in token:
            if string in sr:
                token.remove(string)
    w_count = Counter(x for xs in w_tokens for x in set(xs))
    vectorizer = TfidfVectorizer()
    matrix = vectorizer.fit_transform(w_count)
    # for i, feature in enumerate(vectorizer.get_feature_names()):
        # print(i, feature)
    matrix = matrix.todense()
    df_w = pd.DataFrame(matrix)
    return df_w

# Running Function on Review Text from DataFrame
tokens = tokenizer(df_2['reviewText'][:5])
print(tokens.head())
print(tokens.max())

   0    1    2    3    4    5    6    7    8    9   ...   368  369  370  371  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0  0.0  0.0   

   372  373  374  375  376  377  
0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 378 columns]
0      1.000000
1      1.000000
2      1.000000
3      1.000000
4      1.000000
5      1.000000
6      1.000000
7      1.000000
8      1.000000
9      1.000000
10     1.000000
11     0.730202
12     1.000000
13     1.000000
14     1.000000
15     1.000000
16     1.000000
17     0.312318
18 

In [11]:
# Creating tokens of words from reviews
df_2['tokens'] = df_2['reviewText'].apply(word_tokenize)
df_2.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,tokens
0,0700099867,"[8, 12]",1,Installing the game was a struggle (because of...,"07 9, 2012",A2HD75EMZR8QLN,123,Pay to unlock content? I don't think so.,1341792000,"[Installing, the, game, was, a, struggle, (, b..."
1,0700099867,"[0, 0]",4,If you like rally cars get this game you will ...,"06 30, 2013",A3UR8NLLY1ZHCX,"Alejandro Henao ""Electronic Junky""",Good rally game,1372550400,"[If, you, like, rally, cars, get, this, game, ..."
2,0700099867,"[0, 0]",1,1st shipment received a book instead of the ga...,"06 28, 2014",A1INA0F5CWW3J4,"Amazon Shopper ""Mr.Repsol""",Wrong key,1403913600,"[1st, shipment, received, a, book, instead, of..."
3,0700099867,"[7, 10]",3,"I got this version instead of the PS3 version,...","09 14, 2011",A1DLMTOTHQ4AST,ampgreen,"awesome game, if it did not crash frequently !!",1315958400,"[I, got, this, version, instead, of, the, PS3,..."
4,0700099867,"[2, 2]",4,I had Dirt 2 on Xbox 360 and it was an okay ga...,"06 14, 2011",A361M14PU2GUEG,"Angry Ryan ""Ryan A. Forrest""",DIRT 3,1308009600,"[I, had, Dirt, 2, on, Xbox, 360, and, it, was,..."
5,0700099867,"[0, 0]",4,"Overall this is a well done racing game, with ...","05 11, 2013",A2UTRVO4FDCBH6,A.R.G.,"Good racing game, terrible Windows Live Requir...",1368230400,"[Overall, this, is, a, well, done, racing, gam..."
6,0700099867,"[11, 13]",5,Loved playing Dirt 2 and I thought the graphic...,"08 14, 2011",AN3YYDZAS3O1Y,Bob,A step up from Dirt 2 and that is terrific!,1313280000,"[Loved, playing, Dirt, 2, and, I, thought, the..."
7,0700099867,"[1, 4]",1,I can't tell you what a piece of dog**** this ...,"11 24, 2012",AQTC623NCESZW,Chesty Puller,Crash 3 is correct name AKA Microsoft,1353715200,"[I, ca, n't, tell, you, what, a, piece, of, do..."
8,0700099867,"[0, 1]",4,I initially gave this one star because it was ...,"11 14, 2012",A1QJJU33VNC4S7,D@rkFX,A great game ruined by Microsoft's account man...,1352851200,"[I, initially, gave, this, one, star, because,..."
9,0700099867,"[1, 1]",2,I still haven't figured this one out. Did ever...,"02 8, 2014",A2JLT2WY0F2HVI,D. Sweetapple,Couldn't get this one to work,1391817600,"[I, still, have, n't, figured, this, one, out,..."


In [ ]:
# Want to create TF-IDF Matrix. Need counts of each token per document, that goes into the matrix
# Then make DataFrame of it

In [12]:
# Uploading third Dataset and inspecting its contents
df_3 = pd.DataFrame()
df_3 = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/Managerial_and_Decision_Economics_2013_Video_Games_Dataset.csv', index_col=0)
df_3.head()

,Title,US Sales (millions),Block4,Block2,Block1,Block0.5,YearReleased,2004,2005,2006,...,Tricks,Volleyball,Wakeboarding,Wrestling,FirstPerson,Platform,Isometric,SideScrolling,TopDown,ThirdPerson
Console,,,,,,,,,,,,,,,,,,,,,
Nintendo DS,Super Mario 64 DS,4.69,1,1,1,1,2004,1,0,0,...,0,0,0,0,0,1,0,0,0,1
Sony PSP,Lumines: Puzzle Fusion,0.56,0,0,0,1,2004,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Nintendo DS,WarioWare Touched!,0.54,0,0,0,1,2004,1,0,0,...,0,0,0,0,0,1,1,1,1,1
Sony PSP,Hot Shots Golf: Open Tee,0.49,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,0,0,0,0,1
Nintendo DS,Spider-Man 2,0.45,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,1,0,1,0,1
Nintendo DS,The Urbz: Sims in the City,0.41,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,0,1,0,0,0
Sony PSP,Ridge Racer,0.36,0,0,0,0,2004,1,0,0,...,0,0,0,0,1,0,0,0,0,1
Sony PSP,Metal Gear Ac!d,0.34,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,0,0,0,0,1
Nintendo DS,Madden NFL 2005,0.25,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
# No NaNs in this DataFrame
df_3.isnull().values.any()

False